Install required Libraries

In [ ]:
!pip install wfdb
!pip install kymatio
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 3.1 MB/s eta 0:00:00


Import libraries

In [ ]:
import os
import wfdb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from kymatio.torch import Scattering1D
import torch

#Load data

> Add blockquote



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


A function to load the Electrohysterograph(EHG) signals

In [ ]:
def load_signals(base_folder):
    """
    Loads EHG signals from the specified folder structure.
    Args:
        base_folder: The root folder containing class subfolders.
    Returns:
        signals: List of loaded signals (numpy arrays).
        labels: Corresponding labels of each signal.
    """
    signals = []
    labels = []
    classes = os.listdir(base_folder)  # Class folders

    for label in classes:
        folder_path = os.path.join(base_folder, label)
        if not os.path.isdir(folder_path):
            continue

        for file in os.listdir(folder_path):
            if file.endswith('.dat'):
                file_base = file.split('.')[0]
                dat_path = os.path.join(folder_path, file_base)

                # Load signal using WFDB
                record = wfdb.rdrecord(dat_path)
                signal = record.p_signal  # Extract signals as numpy array

                # Filter only DOCFILT signals
                filtered_indices = [
                    idx for idx, sig in enumerate(record.sig_name) if "DOCFILT" in sig
                ]
                filtered_signal = signal[:, filtered_indices]  # Select filtered signals

                signals.append(filtered_signal)
                labels.append(label)  # Label is the folder name

    return signals, labels


A function to adjust the signal length to match the target length by padding or truncating.

In [ ]:
def adjust_signal_length(signal, target_length):
    """
    Args:
        signal: The input signal (1D numpy array).
        target_length: The desired length (int).

    Returns:
        Adjusted signal of the target length.
    """
    current_length = signal.shape[0]  # The number of samples in the signal
    if current_length > target_length:
        return signal[:target_length]     # Truncate
    elif current_length < target_length:
        padding = target_length - current_length   # Pad with zeros
        return np.pad(signal, (0, padding), mode='constant')
    return signal


Let's now extract features and save to excel file

We are applying a wevelet scattering algorithm

In [6]:
def extract_features_and_save(signals, labels, output_file, scattering):
    """
    Extract features using Wavelet Scattering and save them to an Excel file.

    Args:
        signals: List of signals.
        labels: List of corresponding labels.
        output_file: Path to save the Excel file.
        scattering: Scattering1D object for feature extraction.
    """
    features_list = []
    target_length = scattering.shape  # Get required length for Scattering1D
    target_length = target_length[0]  # Extract as an integer

    for i, (signal, label) in enumerate(zip(signals, labels)):
        print(f"Processing signal {i+1}/{len(signals)}")

        # Adjust signal length for each channel
        signal_tensor = torch.tensor([
            adjust_signal_length(signal[:, ch], target_length) for ch in range(signal.shape[1])
        ], dtype=torch.float32)  # Shape: (channels, target_length)

        # Extract features for each channel
        for ch in range(signal_tensor.shape[0]):
            scattering_features = scattering(signal_tensor[ch, :])
            features_flattened = scattering_features.numpy().flatten()  # Flatten features into a single row

            # Add features and metadata to the list
            features_list.append({
                **{f"feature_{i}": val for i, val in enumerate(features_flattened)},
                "label": label,
                "channel": f"channel_{ch+1}"  # Channel identifier
            })

    df = pd.DataFrame(features_list)
    df.to_excel(output_file, index=False)
    print(f"Features saved to {output_file}")


Main Script

In [8]:

base_folder = '/content/drive/My Drive/EHG'  # Adjust to your folder path
output_excel_file = '/content/drive/My Drive/EHG_Preterm_Birth_Signals_Data_Features.xlsx'

#Load signals and labels
signals, labels = load_signals(base_folder)

#Scattering Network
scattering = Scattering1D(J=5, shape=(2**13,))  # Example: 8192 target length

extract_features_and_save(signals, labels, output_excel_file, scattering)


Processing signal 1/126


<ipython-input-6-da19f83925aa>:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  signal_tensor = torch.tensor([


Processing signal 2/126
Processing signal 3/126
Processing signal 4/126
Processing signal 5/126
Processing signal 6/126
Processing signal 7/126
Processing signal 8/126
Processing signal 9/126
Processing signal 10/126
Processing signal 11/126
Processing signal 12/126
Processing signal 13/126
Processing signal 14/126
Processing signal 15/126
Processing signal 16/126
Processing signal 17/126
Processing signal 18/126
Processing signal 19/126
Processing signal 20/126
Processing signal 21/126
Processing signal 22/126
Processing signal 23/126
Processing signal 24/126
Processing signal 25/126
Processing signal 26/126
Processing signal 27/126
Processing signal 28/126
Processing signal 29/126
Processing signal 30/126
Processing signal 31/126
Processing signal 32/126
Processing signal 33/126
Processing signal 34/126
Processing signal 35/126
Processing signal 36/126
Processing signal 37/126
Processing signal 38/126
Processing signal 39/126
Processing signal 40/126
Processing signal 41/126
Processi